In [51]:
import pandas as pd
import numpy as np

In [2]:
file = '/Users/Lisa/_MLB_Prediction_capstone/cleaned_batting_df.csv'
file2 = '/Users/Lisa/_MLB_Prediction_capstone/cleaned_piching_df.csv'
file3 = '/Users/Lisa/_MLB_Prediction_capstone/cleaned_game_df.csv'

In [3]:
batting_df = pd.read_csv(file)
pitching_df = pd.read_csv(file2)
game_df = pd.read_csv(file3)

In [4]:
batting_df.drop('Unnamed: 0', axis=1, inplace=True)
pitching_df.drop('Unnamed: 0', axis=1, inplace=True)
game_df.drop('Unnamed: 0', axis=1, inplace=True)

## Batting Feature engineering ideas
- All of these stats will be on a PER PLATE APPEARANCE then divide by 162 to get PER GAME STATS
    - Team stats will be summed up and averaged - Will ultimately be compared to a teams 7 day trends history
        - 'totalBases' / total 'plateAppearances' - DONE
        - 'leftonbase'/ total 'plateAppearances' - DONE
        - 'runs' / total 'plateAppearances' - DONE
        - 'strikeOuts' / total 'atBats' - DONE
        - 'baseOnBalls' + 'intentionalWalks' / total 'plateAppearances' - DONE
            - 'hits' / total 'atBats' --- Leverage for game_df. get team stats per game...
            - 'obp' / # of batters on team --- Same as above
        - 'rbi' / total 'atBats' - DONE

In [5]:
# MIA & FLO are different. Standardizing to MIA
batting_df.replace(to_replace='FLA', value='MIA', inplace=True)

In [6]:
# 31 teams showing up since it seems like there are some NaN values for team names. Will deal with should it become an issue...
len(batting_df.teamAbbrev.unique())

31

### Create New Features for TEAM stats

In [7]:
# THIS IS THE FRAMEWORK TO GENERATE TEAM STATISTICS...
# Creating a dataframe containing team annual Batting Averages
a = batting_df.groupby(['year', 'teamAbbrev'])['hits'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
BA = a.join(b, on=['year', 'teamAbbrev'])
BA['team_BA'] = round((BA['hits']/BA['plateAppearances']), 3)
# BA = BA.reset_index()

In [8]:
# Total bases per plate appearance
# Commented out. Taking a different approach
# batting_df['Total_bases_per_plateappearance'] = (batting_df['totalBases'] / batting_df['plateAppearances'])

a = batting_df.groupby(['year', 'teamAbbrev'])['totalBases'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
TB = a.join(b, on=['year', 'teamAbbrev'])
TB['total_bases_avg'] = round((TB['totalBases']/TB['plateAppearances']), 3)
# TB = TB.reset_index()

In [9]:
# LOB per plate appearnace
# Commented out. Taking a different approach
# batting_df['LOB_per_plateappearance'] = (batting_df['leftOnBase'] / batting_df['plateAppearances'])

a = batting_df.groupby(['year', 'teamAbbrev'])['leftOnBase'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
LOB = a.join(b, on=['year', 'teamAbbrev'])
LOB['LOB_avg'] = round((LOB['leftOnBase']/LOB['plateAppearances']), 3)
# LOB = LOB.reset_index()

In [10]:
# Runs per plate appearance
# Commented out. Taking a different approach
#batting_df['runs_per_plateappearance'] = (batting_df['runs'] / batting_df['plateAppearances'])

a = batting_df.groupby(['year', 'teamAbbrev'])['runs'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
RUNS = a.join(b, on=['year', 'teamAbbrev'])
RUNS['runs_avg'] = round((RUNS['runs']/RUNS['plateAppearances']), 3)
# RUNS = RUNS.reset_index()

In [11]:
# K's per AtBat
# Commented out. Taking a different approach
# batting_df['K_per_plateappearance'] = (batting_df['strikeOuts'] / batting_df['atBats'])

a = batting_df.groupby(['year', 'teamAbbrev'])['strikeOuts'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['atBats'].sum()
a = a.to_frame()
b = b.to_frame()
K = a.join(b, on=['year', 'teamAbbrev'])
K['K_avg'] = round((K['strikeOuts']/K['atBats']), 3)
# K = K.reset_index()

In [12]:
# BB per plate appearance
# Commented out. Taking a different approach
# batting_df['BB+IBB_per_plateappearance'] = ((batting_df['baseOnBalls'] + 'took out IBB for simplicity' / batting_df['plateAppearances'])

a = batting_df.groupby(['year', 'teamAbbrev'])['baseOnBalls'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
BB = a.join(b, on=['year', 'teamAbbrev'])
BB['BB_avg'] = round((BB['baseOnBalls']/BB['plateAppearances']), 3)
# BB = BB.reset_index()

In [13]:
# RBI per plate appearance
# Commented out. Taking a different approach
# batting_df['RBI_per_plateappearance'] = (batting_df['rbi'] / batting_df['atBats'])

a = batting_df.groupby(['year', 'teamAbbrev'])['rbi'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
RBI = a.join(b, on=['year', 'teamAbbrev'])
RBI['RBI_avg'] = round((RBI['rbi']/RBI['plateAppearances']), 3)
# RBI = RBI.reset_index()

## Merge all the TEAM stats into one dataframe

In [14]:
team_stats_lst = [BA, TB, LOB, RUNS, K, BB, RBI]

In [15]:
team_stats = pd.concat(team_stats_lst, join='outer', axis=1)

In [16]:
team_stats = team_stats.reset_index()

In [17]:
team_stats.columns

Index(['year', 'teamAbbrev', 'hits', 'plateAppearances', 'team_BA',
       'totalBases', 'plateAppearances', 'total_bases_avg', 'leftOnBase',
       'plateAppearances', 'LOB_avg', 'runs', 'plateAppearances', 'runs_avg',
       'strikeOuts', 'atBats', 'K_avg', 'baseOnBalls', 'plateAppearances',
       'BB_avg', 'rbi', 'plateAppearances', 'RBI_avg'],
      dtype='object')

## Pitching Feature engineering ideas
- Pitching stats will focus on starting pitchers Per9 innings stat (except for certain features)
    - 'winningPercentage'
    - 'runsScoredPer9'
        - 'ops' - may leave out for now...
    - 'hitsPer9'
    - 'era'
    - 'whip'

In [18]:
pitching_df = pitching_df.reindex(columns=['year', 'playerName', 'teamAbbrev', 'winningPercentage',
       'runsScoredPer9', 'hitsPer9', 'strikeoutsPer9', 'baseOnBallsPer9',
       'homeRunsPer9', 'era', 'whip', 'ops', 'gidp', 'avg'])
# Columns to be left out from the reindexing --- 'winPercentage', 'strikeoutsPer9Inn', 'walksPer9Inn', 'hitsPer9Inn'

In [19]:
# MIA & FLO are different. Standardizing to MIA
pitching_df.replace(to_replace='FLA', value='MIA', inplace=True)

In [20]:
# 31 teams showing up since it seems like there are some NaN values for team names. Will deal with should it become an issue...
len(pitching_df.teamAbbrev.unique())

31

## Game DF Feature ideas
- Turn VisitorRunsScored & HomeRunsScore into a column that indicates if the home team won - DONE.
- Combine VisitorHBP & VisitorBB & VisitorIBB into a single column containing all the stats - DONE.
- Same with HomeHBP & HomeBB & HomeIBB - DONE.
<br/><br/>
- Combine VisitorH / VisitorAB = team_BA - DONE.
- Same with HomeH / HomeAB - DONE. 
<br/><br/>
- Eventually add on all the other engineered features into this dataframe...
<br/><br/>
- Since the game_df has a date column, I'd like to leverage that to look at 7-day-trends, 14-day-trends, etc

In [21]:
# Creating a Target for the model to eventually train on
game_df['Home_team_won?'] = game_df['HomeRunsScore'] > game_df['VisitorRunsScored']

In [22]:
# game_df['Visitor_HBP+BB+IBB'] = game_df['VisitorHBP'] + game_df['VisitorBB'] + game_df['VisitorIBB']
# game_df['Home_HBP+BB+IBB'] = game_df['HomeHBP'] + game_df['HomeBB'] + game_df['HomeIBB']

# Commenting out for now until I figure out if I need to drop these or not...
# game_df.drop(['VisitorHBP', 'VisitorBB', 'VisitorIBB', 'HomeHBP', 'HomeBB', 'HomeIBB'], axis=1, inplace=True)

In [23]:
# Adding a Home & Away BA per game to compare batting trends over time
game_df['Visitor_BA'] = round((game_df['VisitorH'] / game_df['VisitorAB']), 3)
game_df['Home_BA'] = round((game_df['HomeH'] / game_df['HomeAB']), 3)

In [24]:
# Creating a new column in order to add Date/Time to my dataframe
game_df['New_Date'] = pd.to_datetime(game_df['Date'].astype(str), format='%Y%m%d')

In [25]:
# Rearranging columns so new date/time is at the front of the DF
game_df = game_df.reindex(columns=['New_Date','Date', 'VisitingTeam','VisitorStartingPitcherName',
                                 'HomeTeam','HomeStartingPitcherName', 'VisitorRunsScored',
       'HomeRunsScore', 'VisitorAB', 'VisitorH','Visitor_BA', 'VisitorHBP', 'VisitorBB',
       'VisitorIBB', 'VisitorLOB', 'HomeAB', 'HomeH', 'Home_BA', 'HomeHBP', 'HomeBB', 'HomeIBB', 'HomeLOB', 'Home_team_won?'])
game_df.drop('Date', axis='columns', inplace=True)

In [26]:
# Had an extra team due to the Marlins having two different labels. Updating their team name to MIA
game_df.replace(to_replace='FLO', value='MIA', inplace=True)
# Updating game_df so team abbreviations match. Will be leveraged when aggregating teams stats
game_df.replace(to_replace={"NYA":"NYY", "SDN":"SD", "CHN":"CHC", "SLN":"STL", "SFN":"SF", "LAN":"LAD", "TBA":"TB", "KCA":"KC", "CHA":"CWS", "ANA":"LAA", "NYN":"NYM"}, inplace=True)

In [27]:
# game_df has 30 teams. Perfect!
len(game_df.VisitingTeam.unique())

30

In [28]:
# How to Access the
game_df['prior_year'] = game_df['New_Date'].dt.year-1

In [29]:
game_df['prior_year'].dtypes

dtype('int64')

In [30]:
game_df.shape

(24297, 23)

In [31]:
game_df = game_df.merge(pitching_df, how='left', left_on=['prior_year','VisitorStartingPitcherName'], right_on=['year', 'playerName'])

In [32]:
game_df.columns, game_df.shape

(Index(['New_Date', 'VisitingTeam', 'VisitorStartingPitcherName', 'HomeTeam',
        'HomeStartingPitcherName', 'VisitorRunsScored', 'HomeRunsScore',
        'VisitorAB', 'VisitorH', 'Visitor_BA', 'VisitorHBP', 'VisitorBB',
        'VisitorIBB', 'VisitorLOB', 'HomeAB', 'HomeH', 'Home_BA', 'HomeHBP',
        'HomeBB', 'HomeIBB', 'HomeLOB', 'Home_team_won?', 'prior_year', 'year',
        'playerName', 'teamAbbrev', 'winningPercentage', 'runsScoredPer9',
        'hitsPer9', 'strikeoutsPer9', 'baseOnBallsPer9', 'homeRunsPer9', 'era',
        'whip', 'ops', 'gidp', 'avg'],
       dtype='object'),
 (24299, 37))

In [33]:
game_df = game_df.merge(pitching_df, how='left', left_on=['prior_year','HomeStartingPitcherName'], right_on=['year', 'playerName'])

In [34]:
game_df.columns, game_df.shape

(Index(['New_Date', 'VisitingTeam', 'VisitorStartingPitcherName', 'HomeTeam',
        'HomeStartingPitcherName', 'VisitorRunsScored', 'HomeRunsScore',
        'VisitorAB', 'VisitorH', 'Visitor_BA', 'VisitorHBP', 'VisitorBB',
        'VisitorIBB', 'VisitorLOB', 'HomeAB', 'HomeH', 'Home_BA', 'HomeHBP',
        'HomeBB', 'HomeIBB', 'HomeLOB', 'Home_team_won?', 'prior_year',
        'year_x', 'playerName_x', 'teamAbbrev_x', 'winningPercentage_x',
        'runsScoredPer9_x', 'hitsPer9_x', 'strikeoutsPer9_x',
        'baseOnBallsPer9_x', 'homeRunsPer9_x', 'era_x', 'whip_x', 'ops_x',
        'gidp_x', 'avg_x', 'year_y', 'playerName_y', 'teamAbbrev_y',
        'winningPercentage_y', 'runsScoredPer9_y', 'hitsPer9_y',
        'strikeoutsPer9_y', 'baseOnBallsPer9_y', 'homeRunsPer9_y', 'era_y',
        'whip_y', 'ops_y', 'gidp_y', 'avg_y'],
       dtype='object'),
 (24300, 51))

In [37]:
for name in game_df.columns:
    print(name)

New_Date
VisitingTeam
VisitorStartingPitcherName
HomeTeam
HomeStartingPitcherName
VisitorRunsScored
HomeRunsScore
VisitorAB
VisitorH
Visitor_BA
VisitorHBP
VisitorBB
VisitorIBB
VisitorLOB
HomeAB
HomeH
Home_BA
HomeHBP
HomeBB
HomeIBB
HomeLOB
Home_team_won?
prior_year
year_x
playerName_x
teamAbbrev_x
winningPercentage_x
runsScoredPer9_x
hitsPer9_x
strikeoutsPer9_x
baseOnBallsPer9_x
homeRunsPer9_x
era_x
whip_x
ops_x
gidp_x
avg_x
year_y
playerName_y
teamAbbrev_y
winningPercentage_y
runsScoredPer9_y
hitsPer9_y
strikeoutsPer9_y
baseOnBallsPer9_y
homeRunsPer9_y
era_y
whip_y
ops_y
gidp_y
avg_y


### Team Batting stats to add to DF

In [35]:
team_stats.head()

,year,teamAbbrev,hits,plateAppearances,team_BA,totalBases,plateAppearances,total_bases_avg,leftOnBase,plateAppearances,...,runs_avg,strikeOuts,atBats,K_avg,baseOnBalls,plateAppearances,BB_avg,rbi,plateAppearances,RBI_avg
0,2009,ARI,1310,5858,0.224,2195,5858,0.375,2225,5858,...,0.116,1224,5182,0.236,545,5858,0.093,664,5858,0.113
1,2009,ATL,1492,6510,0.229,2336,6510,0.359,2432,6510,...,0.119,1137,5678,0.200,637,6510,0.098,723,6510,0.111
2,2009,BAL,1367,5610,0.244,2107,5610,0.376,2132,5610,...,0.120,913,5073,0.180,449,5610,0.080,628,5610,0.112
3,2009,BOS,1915,8245,0.232,3151,8245,0.382,3223,8245,...,0.128,1435,7202,0.199,864,8245,0.105,1036,8245,0.126
4,2009,CHC,1392,6214,0.224,2228,6214,0.359,2458,6214,...,0.113,1176,5459,0.215,590,6214,0.095,679,6214,0.109


In [38]:
team_stats.drop('plateAppearances', axis=1, inplace=True)

In [43]:
team_stats.columns, team_stats.shape

(Index(['year', 'teamAbbrev', 'hits', 'team_BA', 'totalBases',
        'total_bases_avg', 'leftOnBase', 'LOB_avg', 'runs', 'runs_avg',
        'strikeOuts', 'atBats', 'K_avg', 'baseOnBalls', 'BB_avg', 'rbi',
        'RBI_avg'],
       dtype='object'),
 (330, 17))

In [45]:
game_df = game_df.merge(team_stats, how='left', left_on=['prior_year', 'VisitingTeam'], right_on=['year', 'teamAbbrev'])

In [46]:
game_df = game_df.merge(team_stats, how='left', left_on=['prior_year', 'HomeTeam'], right_on=['year', 'teamAbbrev'])

In [50]:
for name in game_df.columns:
    print(name)

New_Date
VisitingTeam
VisitorStartingPitcherName
HomeTeam
HomeStartingPitcherName
VisitorRunsScored
HomeRunsScore
VisitorAB
VisitorH
Visitor_BA
VisitorHBP
VisitorBB
VisitorIBB
VisitorLOB
HomeAB
HomeH
Home_BA
HomeHBP
HomeBB
HomeIBB
HomeLOB
Home_team_won?
prior_year
year_x
playerName_x
teamAbbrev_x
winningPercentage_x
runsScoredPer9_x
hitsPer9_x
strikeoutsPer9_x
baseOnBallsPer9_x
homeRunsPer9_x
era_x
whip_x
ops_x
gidp_x
avg_x
year_y
playerName_y
teamAbbrev_y
winningPercentage_y
runsScoredPer9_y
hitsPer9_y
strikeoutsPer9_y
baseOnBallsPer9_y
homeRunsPer9_y
era_y
whip_y
ops_y
gidp_y
avg_y
year_x
teamAbbrev_x
hits_x
team_BA_x
totalBases_x
total_bases_avg_x
leftOnBase_x
LOB_avg_x
runs_x
runs_avg_x
strikeOuts_x
atBats_x
K_avg_x
baseOnBalls_x
BB_avg_x
rbi_x
RBI_avg_x
year_y
teamAbbrev_y
hits_y
team_BA_y
totalBases_y
total_bases_avg_y
leftOnBase_y
LOB_avg_y
runs_y
runs_avg_y
strikeOuts_y
atBats_y
K_avg_y
baseOnBalls_y
BB_avg_y
rbi_y
RBI_avg_y


In [56]:
game_df['current_year'] = game_df['New_Date'].dt.year

### Next steps, feature engineer 7-day trends for key statistics

## Train and fit a model to see how these feature preform so far...

In [ ]:
y = game_df['Home_team_won?']

In [ ]:
X = game_df.drop(['VisitorRunsScored', 'HomeRunsScore', 'Home_team_won?', 
'VisitorAB',
'VisitorH',
'Visitor_BA',
'VisitorHBP',
'VisitorBB',
'VisitorIBB',
'VisitorLOB',
'HomeAB',
'HomeH',
'Home_BA',
'HomeHBP',
'HomeBB',
'HomeIBB',
'HomeLOB'], axis=1)

train on 2 years of data and then test on the following season (through all seasons)

In [ ]:
def my_train_test_split(X, y):
    '''
    creating a custom train_test_split in order to split data by seasons
    i.e.
    - train on 2010-2016 seasons
    - test on 2018-2019 seasons
    
    Parameters:
    X: features
    y: target value
    '''
    years = np.arange(2010,2018)
    mask = game_df['current_year'].isin(years)
    X_train = X[mask]
    X_test = X[~mask]
    y_train = y[mask]
    y_test = y[~mask]
    return X_train, X_test, y_train, y_test